In [ ]:
# ==== Generate LUT.csv for TPA calculations ====
import numpy as np
import pandas as pd
from scipy.constants import c, hbar

def generate_tpa_lut(NAv, nr, tau, Epulse, dcf, R, H, 
                    r_range=(0, 20e-6, 201),  # r in meters, 201 points
                    z_range=(-500e-6, 500e-6, 501)):  # z in meters, 501 points
    
    # Constants
    tpalength = 405e-9  # wavelength in meters
    tpatau = 160e-15    # pulse duration in seconds
    b = 1.5e-11        # beta_2 in m/W
    f = c/tpalength    # frequency in Hz
    
    # Create coordinate grids
    r = np.linspace(r_range[0], r_range[1], r_range[2])
    z = np.linspace(z_range[0], z_range[1], z_range[2])
    R_mesh, Z_mesh = np.meshgrid(r, z)
    
    # Calculate beam waist
    waist = np.sqrt((tpalength/(np.pi*NAv))**2 + (NAv*Z_mesh/nr)**2)
    
    # Calculate TPA rate
    ntpa = ((2*np.sqrt(np.log(4)))/(2*np.pi**3.5) * 
            (Epulse**2*b/(tau*hbar*f)) * 
            (1/waist**4) * 
            np.exp(-4*R_mesh**2/waist**2))
    
    # Calculate interference term
    nint = (np.sqrt(ntpa * np.roll(ntpa, int(2*H/(z[1]-z[0])), axis=0)) * 
            dcf * 2 * R)
    
    # Calculate reflection term
    nref = np.roll(ntpa, int(2*H/(z[1]-z[0])), axis=0) * R**2
    
    # Calculate total rate
    nsum = ntpa + nref + nint
    
    # Calculate final density
    ndia = nsum * 2 * np.pi * R_mesh
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame({
        'r': R_mesh.flatten(),
        'z': Z_mesh.flatten(),
        'ndia': ndia.flatten()
    })
    
    # Save to CSV
    df.to_csv('Garfield_workplace/LUT.csv', index=False)
    print("LUT.csv generated successfully in Garfield_workplace/")
    
    return df

# Example usage with default parameters
NAv = 0.5      # Numerical aperture
nr = 2.4582    # Refractive index
tau = 160e-15  # Pulse duration in seconds
Epulse = 1e-9  # Pulse energy in Joules
dcf = 0.5      # Delay factor
Ref = 0.8        # Reflection coefficient
Hcenter = 50e-6      # Center position in meters (z=0: bottom surf; z=zmax) 

# Generate LUT
lut_df = generate_tpa_lut(NAv, nr, tau, Epulse, dcf, Ref, Hcenter)

In [ ]:
# ==== Clean and copy files from Garfield_workplace to Garfield_satellite ====
import shutil
import os

# Clean Garfield_satellite directory
if os.path.exists("Garfield_satellite"):
    shutil.rmtree("Garfield_satellite")
    print("Cleaned Garfield_satellite directory")

# Create fresh Garfield_satellite directory
os.makedirs("Garfield_satellite")

# Files to copy from Garfield_workplace
files_to_copy = ["Diamond_4p.C", "CMakeLists.txt", "transfer.csv"]

# Copy each file from Garfield_workplace
for file in files_to_copy:
    source_path = os.path.join("Garfield_workplace", file)
    if os.path.exists(source_path):
        shutil.copy2(source_path, os.path.join("Garfield_satellite", file))
        print(f"Copied {file} from Garfield_workplace to Garfield_satellite/")
    else:
        print(f"Warning: {file} not found in Garfield_workplace")

In [ ]:
import paramiko
from scp import SCPClient

# ==== Variables to configure ====
num_evt_perjob = 1000
num_job = 55

local_file = "Garfield_satellite"  # Local file path
remote_path = "/tmp/"      # Remote target path (on server)
bash_command = f"cd /tmp/Garfield_satellite/build && cmake .. && make && source submit.sh {num_evt_perjob} {num_job}"  # Command to execute

server = "tcad01.hep.manchester.ac.uk"     # Remote host
username = "lihuazhen"     # Username
# If using SSH key (default ~/.ssh/id_rsa), password is not required

# ==== Establish SSH connection ====
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(server, username=username)

# ==== Upload file ====
with SCPClient(ssh.get_transport()) as scp:
    scp.put(local_file, remote_path)

# ==== Execute remote command ====
stdin, stdout, stderr = ssh.exec_command(bash_command)

# ==== Print output results ====
print(">> STDOUT:")
print(stdout.read().decode())

print(">> STDERR:")
print(stderr.read().decode())

ssh.close()
